Course Instructor: Bernd Neumayr, JKU

# UE05: SHACL

Complete the **10 tasks (1 point per task)** in the `4. SHACL` sheet of `SemAI.jar` first and then transfer them to this notebook.

For each task include:
- A headline including the task number
- The task description 
- The data graph and your solution (the shapes graph) in executable form
- After executing the validation, print out the validation results in tabular form.  


## Preparations

install and import rdflib and pyshacl

In [30]:
# Install required packages in the current Jupyter kernel
!pip install -q rdflib 
!pip3 install -q pyshacl

In [31]:
# Imports
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace, Dataset
from rdflib.namespace import FOAF , XSD , RDFS 
from rdflib.plugins.sparql.processor import SPARQLResult
from rdflib.namespace import NamespaceManager

from pyshacl import validate

import pandas as pd

def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = [ { var : res[var].n3(graph.namespace_manager) if (isinstance(res[var],URIRef) and use_prefixes) else res[var] for var in results.vars } for res in results ]     
                                        # construct a list of dictionaries, as intermediate format to construct the pandas DataFrame, use prefixes to abbreviate URIs                
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      

def validation_report_as_dataframe(validation_report):
  df = sparql_select(results_graph,"""
		SELECT  ?focusNode ?resultPath ?value ?sourceConstraintComponent ?sourceShape ?resultMessage
		WHERE
  		{ ?vr	a sh:ValidationResult ;
						sh:focusNode ?focusNode ;
						sh:sourceConstraintComponent ?sourceConstraintComponent ;
						sh:sourceShape ?sourceShape ;
						sh:resultMessage ?resultMessage .					 
				OPTIONAL { ?vr sh:value ?value . }
				OPTIONAL { ?vr sh:resultPath ?resultPath . }
  		}
  """,use_prefixes=True)
  return df

def shacl_validate(dg,sg):
  return validate(dg,shacl_graph=sg,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=False,
      js=False,
      debug=False)  
  

def shacl_validate_with_rules(dg,sg):
	return validate(dg,shacl_graph=sg,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=True,
      iterate_rules=True, inplace=True,
      js=False,
      debug=False)


#Task 1

Wenn eine Person eine `x:knows` Beziehung hat, dann sollte das Ziel dieser Beziehung als IRI angegeben sein. 

In [32]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/>
BASE         <http://example.org/>

<Peter>  rdf:type  <Person> ;
        :knows    <John> ;
        :knows    [ :name  "Peter" ] .

<John>  :knows  [ :name   "Mary" ] .   
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/>
BASE <http://example.org/>

<PersonShape> a sh:NodeShape ;
        sh:targetClass <Person> ; 
        sh:property <PersonKnowsShape> .

<PersonKnowsShape> 
  sh:path :knows ;
  sh:nodeKind sh:IRI ;
  sh:message "Value is not Node Kind IRI" .
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  
validation_report_as_dataframe(results_graph) 

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Peter,:knows,n2e8dc827e11b4de48f696326ed505885b1,sh:NodeKindConstraintComponent,:PersonKnowsShape,Value is not Node Kind IRI


#Task 2

Die Klassen `Man` und `Woman` sind disjunkt. 

Hinweis: verwenden Sie `sh:not`. 

In [33]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/>
BASE         <http://example.org/>

<YoungMan>  rdfs:subClassOf  <Man> .

<John>  rdf:type  <YoungWoman> , <YoungMan> .

<Mary>  rdf:type  <Woman> .

<Peter>  rdf:type  <Man> .

<YoungWoman>  rdfs:subClassOf  <Woman> .

<Jane>  rdf:type  <YoungWoman> .

<Susi>  rdf:type  <YoungWoman> , <Man> .  
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/>
BASE         <http://example.org/>

<ManShape> a sh:NodeShape ;
    sh:targetClass <Man> ; 
    sh:not [ sh:class <Woman> ] .
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  
validation_report_as_dataframe(results_graph) 

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Susi,None,:Susi,sh:NotConstraintComponent,:ManShape,Node :Susi conforms to shape [ sh:class :Woman ]
1,:John,None,:John,sh:NotConstraintComponent,:ManShape,Node :John conforms to shape [ sh:class :Woman ]


#Task 3
Wenn eine Person eine `:knows` Beziehung hat, dann muss das Ziel dieser Beziehung eine Person sein.



In [34]:

dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/>
BASE         <http://example.org/>

<Peter>  rdf:type  <Person> ;
        :knows    <Mary> ;
        :knows    [ :name  "Mary" ] ;
        :knows    [ rdf:type  <Person> ] .

<Jane>  :knows  <Jim> .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/>
BASE         <http://example.org/>

<PersonKnowsShape> 
    sh:targetClass <Person> ;
    sh:path :knows ;
    sh:class <Person> .
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  
validation_report_as_dataframe(results_graph) 

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Peter,:knows,:Mary,sh:ClassConstraintComponent,:PersonKnowsShape,Value does not have class :Person
1,:Peter,:knows,n48db11c5330b4c52b4eac5e7088cfb35b1,sh:ClassConstraintComponent,:PersonKnowsShape,Value does not have class :Person


#Task 4
Jede Person hat genau ein Alter (`:age`). Eine Person ist mindestens 0 und maximal 150 Jahre alt. Erwachsene (Klasse `Adult`) sind mindestens 19 Jahre alt. Senioren sind mindestens 65 Jahre alt. Kinder sind maximal 12 Jahre alt.
Verwenden Sie `sh:minInclusive` und `sh:maxInclusive` (siehe SHACL Recommendation).


In [35]:

dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/>
BASE         <http://example.org/>

<ABC>   :age    -1 , 27 , 197 .

<John>  rdf:type  <Person> ;
        :age      151 .

<Adult>  rdfs:subClassOf  <Person> .

<Beny>  rdf:type  <Child> ;
        :age      13 .

<Zoe>   rdf:type  <Child> ;
        :age      12 .

<Mary>  rdf:type  <Woman> ;
        :age      17 .

<Bibi>  rdf:type  <Person> ;
        :age      0 .

<Peter>  rdf:type  <Person> .

<Senior>  rdfs:subClassOf  <Adult> .

<Jane>  rdf:type  <Woman> ;
        :age      67 .

<Lili>  rdf:type  <Person> ;
        :age      -1 .

<Senior1>  rdf:type  <Senior> ;
        :age      65 .

<Child>  rdfs:subClassOf  <Person> .

<Bibo>  rdf:type  <Person> ;
        :age      34 , 12 .

<Woman>  rdfs:subClassOf  <Adult> .

<Senior2>  rdf:type  <Senior> ;
        :age      64 .

""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/>
BASE         <http://example.org/>

<PersonAgeShape> 
  sh:targetClass <Person> ;
  sh:path :age ;
  sh:minInclusive 0 ;
  sh:maxInclusive 150 ;
  sh:minCount 1 ;
  sh:maxCount 1 . 

<AdultAgeShape> 
  sh:targetClass <Adult> ;
  sh:path :age ;
  sh:minInclusive 19 .

<SeniorAgeShape> 
  sh:targetClass <Senior> ;
  sh:path :age ;
  sh:minInclusive 65 .

<ChildAgeShape> 
  sh:targetClass <Child> ;
  sh:path :age ;
  sh:maxInclusive 12 . 

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  
validation_report_as_dataframe(results_graph) 

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Mary,:age,17,sh:MinInclusiveConstraintComponent,:AdultAgeShape,"Value is not >= Literal(""19"", datatype=xsd:int..."
1,:Beny,:age,13,sh:MaxInclusiveConstraintComponent,:ChildAgeShape,"Value is not <= Literal(""12"", datatype=xsd:int..."
2,:Lili,:age,-1,sh:MinInclusiveConstraintComponent,:PersonAgeShape,"Value is not >= Literal(""0"", datatype=xsd:inte..."
3,:John,:age,151,sh:MaxInclusiveConstraintComponent,:PersonAgeShape,"Value is not <= Literal(""150"", datatype=xsd:in..."
4,:Peter,:age,None,sh:MinCountConstraintComponent,:PersonAgeShape,Less than 1 values on :Peter->:age
5,:Bibo,:age,None,sh:MaxCountConstraintComponent,:PersonAgeShape,More than 1 values on :Bibo->:age
6,:Senior2,:age,64,sh:MinInclusiveConstraintComponent,:SeniorAgeShape,"Value is not >= Literal(""65"", datatype=xsd:int..."


#Task 5
Jede Person hat maximal einen Namen. Der Name muss ein Literal sein und vom Datentyp String. Personen dürfen abgesehen von `:name` und `rdf:type` keine weiteren Eigenschaften haben.


In [36]:

dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/>
BASE         <http://example.org/>

<John>  rdf:type  <MalePerson> ;
        :knows    <Mary> .

<FemalePerson>  rdfs:subClassOf  <Person> .

<Mary>  rdf:type  <FemalePerson> ;
        :name     "Mary" .

<Bibi>  rdf:type  <Person> ;
        :name     "Jim" , "Bibi" .

<Jane>  rdf:type  <Person> ;
        :name     2343 .

<Jim>   rdf:type  <Person> ;
        :name     <Jim> .

<MalePerson>  rdfs:subClassOf  <Person> .

""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/>
BASE         <http://example.org/>

   <PersonShape> a sh:NodeShape ;
     sh:targetClass <Person> ;
     sh:closed true; 
     sh:ignoredProperties (rdf:type) ;
     sh:property <NameShape> .   

   <NameShape> 
     sh:path :name ;
     sh:maxCount 1 ;
     sh:nodeKind sh:Literal ;
     sh:datatype xsd:string .   

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  
validation_report_as_dataframe(results_graph) 

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:John,:knows,:Mary,sh:ClosedConstraintComponent,:PersonShape,Node :John is closed. It cannot have value: :Mary
1,:Jim,:name,:Jim,sh:NodeKindConstraintComponent,:NameShape,Value is not of Node Kind sh:Literal
2,:Jim,:name,:Jim,sh:DatatypeConstraintComponent,:NameShape,Value is not Literal with datatype xsd:string
3,:Bibi,:name,None,sh:MaxCountConstraintComponent,:NameShape,More than 1 values on :Bibi->:name
4,:Jane,:name,2343,sh:DatatypeConstraintComponent,:NameShape,Value is not Literal with datatype xsd:string


#Task 6

Städte sind via Property `:inCountry` Ländern zugeordnet. Europäische Städte sind europäischen Ländern zugeordnet. Österreichische Städte sind `Austria` zugeordnet.


In [37]:

dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/>
BASE         <http://example.org/>

<Paris>  rdf:type   <EuropeanCity> ;
        :inCountry  <France> .

<AustrianCity>  rdfs:subClassOf  <EuropeanCity> .

<Salzburg>  rdf:type  <AustrianCity> ;
        :inCountry  <Germany> .

<EuropeanCountry>  rdfs:subClassOf  <Country> .

<NewYork>  rdf:type  <City> ;
        :inCountry  <USA> .

<Germany>  rdf:type  <EuropeanCountry> .

<EuropeanCity>  rdfs:subClassOf  <City> .

<Vienna>  rdf:type  <AustrianCity> ;
        :inCountry  <Austria> .

<Austria>  rdf:type  <EuropeanCountry> .

""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/>
BASE         <http://example.org/>

   <CityInCountryShape> 
     sh:targetClass <City> ;
     sh:path :inCountry ;
     sh:class <Country> .

   <EuropeanCityInCountryShape> 
     sh:targetClass <EuropeanCity> ;
     sh:path :inCountry ;
     sh:class <EuropeanCountry> .

   <AustrianCityInCountryShape> 
     sh:targetClass <AustrianCity> ;
     sh:path :inCountry ;
     sh:in ( <Austria> ) .   


""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  
validation_report_as_dataframe(results_graph) 

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Paris,:inCountry,:France,sh:ClassConstraintComponent,:EuropeanCityInCountryShape,Value does not have class :EuropeanCountry
1,:Paris,:inCountry,:France,sh:ClassConstraintComponent,:CityInCountryShape,Value does not have class :Country
2,:NewYork,:inCountry,:USA,sh:ClassConstraintComponent,:CityInCountryShape,Value does not have class :Country
3,:Salzburg,:inCountry,:Germany,sh:InConstraintComponent,:AustrianCityInCountryShape,Value :Germany not in list [':Austria']


#Task 7
Die Property `:worksFor` darf nur Personen (als Subjekt) und Organisation (als Objekt) miteinander verbinden.

Hinweis: Lösen Sie die Aufgabe ohne Property Shapes.

In [38]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/>
BASE         <http://example.org/>

<Mary>  :worksFor  <JKU> .

<Jane>  rdf:type   <Person> ;
        :worksFor  <LMU> .

<Jim>   rdf:type   <Man> ;
        :worksFor  <LMU> .

<University>  rdfs:subClassOf  <Organization> .

<Bob>   rdf:type   <Person> ;
        :worksFor  [ rdf:type  <Organization> ] .

<JKU>   rdf:type  <University> .

""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/>
BASE         <http://example.org/>

  <WorksForSubjectsShape> 
    sh:targetSubjectsOf :worksFor ;
    sh:class <Person> .

  <WorksForTargetsShape> 
    sh:targetObjectsOf :worksFor ;
    sh:class <Organization> .

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  
validation_report_as_dataframe(results_graph) 

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:LMU,None,:LMU,sh:ClassConstraintComponent,:WorksForTargetsShape,Value does not have class :Organization
1,:Jim,None,:Jim,sh:ClassConstraintComponent,:WorksForSubjectsShape,Value does not have class :Person
2,:Mary,None,:Mary,sh:ClassConstraintComponent,:WorksForSubjectsShape,Value does not have class :Person


#Task 8
`Peter` kennt ausschließlich Männer, die in Österreich leben.


In [39]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/>
BASE         <http://example.org/>

<John>  rdf:type  <Man> ;
        :livesIn  <Austria> .

<Mary>  rdf:type  <Woman> ;
        :knows    <Jim> ;
        :livesIn  <Austria> .

<Hans>  rdf:type  <Man> .

<Peter>  :knows  <Josef> , <Hans> , <Franz> , <John> , <Mary> .

<Franz>  rdf:type  <Man> ;
        :livesIn  <Germany> .

""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/>
BASE         <http://example.org/>

    <PeterShape>
        sh:targetNode <Peter> ;
        sh:property <PeterKnowsShape> .
        
    <PeterKnowsShape>
        sh:path :knows ;
        sh:node <PeterKnowsNodeShape> . 
        
    <PeterKnowsNodeShape>
        sh:class <Man> ;
        sh:property <PeterKnowsLivesShape> .
    
    <PeterKnowsLivesShape>
        sh:path :livesIn ;
        sh:minCount 1 ;
        sh:in (<Austria>) .   

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  
validation_report_as_dataframe(results_graph) 

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Peter,:knows,:Franz,sh:NodeConstraintComponent,:PeterKnowsShape,Value does not conform to Shape :PeterKnowsNod...
1,:Peter,:knows,:Hans,sh:NodeConstraintComponent,:PeterKnowsShape,Value does not conform to Shape :PeterKnowsNod...
2,:Peter,:knows,:Josef,sh:NodeConstraintComponent,:PeterKnowsShape,Value does not conform to Shape :PeterKnowsNod...
3,:Peter,:knows,:Mary,sh:NodeConstraintComponent,:PeterKnowsShape,Value does not conform to Shape :PeterKnowsNod...


#Task 9
Das Nettogewicht eines Produkts darf nicht größer sein als das Bruttogewicht.
Verwenden Sie `sh:lessThanOrEquals` (siehe SHACL Recommendation).

In [43]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/>
BASE         <http://example.org/>

<Cookies>  rdf:type   <Product> ;
        :grossWeight  0.2 ;
        :netWeight    0.12 .

<Milk>  rdf:type      <Product> ;
        :grossWeight  1.1 ;
        :netWeight    1 .

<Peter>  rdf:type     <Person> ;
        :grossWeight  74 ;
        :netWeight    72 .

<Bread>  rdf:type     <Product> ;
        :grossWeight  1.1 ;
        :netWeight    1.2 .

""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/>
BASE         <http://example.org/>

   
<ProductNetWeightShape>  
      sh:targetClass <Product> ;
      sh:path :netWeight ;
      sh:lessThanOrEquals :grossWeight .     

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  
validation_report_as_dataframe(results_graph) 

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Bread,:netWeight,1.2,sh:LessThanOrEqualsConstraintComponent,:ProductNetWeightShape,"Value of :Bread->:grossWeight < Literal(""1.2"",..."


#Task 10

Die Objekte von knows-Statements haben einen Namen (Property :name) oder haben einen Vornamen (:givenName) und einen Nachnamen (:familyName). eine
  

In [41]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/>
BASE         <http://example.org/>

<Sara>  :givenName  "Sarah" .

<John>  :familyName  "Black" .

<Mary>  :knows  <Pete> .

<Bob>   :familyName  "Builder" ;
        :givenName   "Bob" .

<Pete>  :knows  <Bob> , <Sara> , <John> ;
        :name   "Peter Parker" .

""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sh:   <http://www.w3.org/ns/shacl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX :     <http://example.org/>
BASE         <http://example.org/>

<KnowsShape>
  sh:targetObjectsOf :knows ;
  sh:or (
    [ sh:property [ sh:path :name; sh:minCount 1 ] ]
    [ sh:property [ sh:path :givenName; sh:minCount 1 ]; sh:property [ sh:path :familyName; sh:minCount 1 ]]
  ) . 

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  
validation_report_as_dataframe(results_graph) 

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:John,None,:John,sh:OrConstraintComponent,:KnowsShape,Node :John does not conform to one or more sha...
1,:Sara,None,:Sara,sh:OrConstraintComponent,:KnowsShape,Node :Sara does not conform to one or more sha...
